In [1]:
import datetime
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

import csv

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode

from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome

options = Options()
options.headless = True
service = Service('/Users/williamfoote/Downloads/chromedriver-mac-arm64/chromedriver')
browser = Chrome(service=service, options=options)

pd.set_option('display.max_rows', 1000)

In [2]:
#Returns the difference between the max and the minimum of a set of values
def max_minus_min(x): return max(x) - min(x)
#Returns the difference between the last and the average of a set of values
def last_minus_avg(x): return last(x) - mean(x)

In [3]:
# #in this function, we pass in a url that we wills scrape and a csv file that will be used to store/retrieve data
# def bovada_scrape(url, file):
#     #initiate selenium, you'll need to point to a local chromedriver you've downloaded
#     browser = webdriver.Chrome(service=service, options=options)
#     browser.get(url)
#     browser.implicitly_wait(10)

#     #here we are retrieving elements from the page based on their class name
#     title = browser.find_elements(By.CLASS_NAME, 'market-header')
#     heading = browser.find_elements(By.CLASS_NAME, 'game-heading')
#     sub_title = browser.find_elements(By.CLASS_NAME, 'market-name')
#     outcomes = browser.find_elements(By.CLASS_NAME, 'outcomes')
#     bet_price = browser.find_elements(By.CLASS_NAME, 'bet-price')
#     #storing current timestamp so we can see data over time
#     now = datetime.datetime.now()

#     #checking to see if there is an input file. There should be, unless this is the first time checking     
#     try:
#         in_df = pd.read_csv(file)
#     except Exception:
#         try:
#             del in_df
#         except Exception:
#             pass

#     #initiating lists for the data we want...outcome, bet_price and title
#     o = []
#     b = []
#     t = []
#     n = []
#     #for all the objects returned by selenium, will add them to lists
#     for i in range(len(outcomes)):
#         o.append(outcomes[i].text)
#         b.append(bet_price[i].text)
#     for i in range(len(title)):
#         t.append(title[i].text)
#         n.append(sub_title[i].text)
#     #create a dataframe from the lists
#     df = pd.DataFrame({'outcomes':o,'bet_price':b, 'title':t, 'market_name':n})
#     #add a field for the current timestamp
#     df['date'] = now
#     #replace "EVEN" with zero and store data as a float
#     df['bet_price']=df['bet_price'].replace('EVEN', '0')
#     # Replace empty strings and other non-numeric placeholders with 0.0
#     df['bet_price'] = df['bet_price'].replace(['', '-', 'N/A', 'EVEN'], '0.0')

#     # Now convert to float
#     df['bet_price'] = df['bet_price'].astype(float)

#     #append this dataframe we created with our historical data
#     try:
#         df = in_df.append(df)
#     except Exception:
#         pass

#     #save off the file, overwriting the existing
#     df.to_csv(file, index = None)
#     df = pd.read_csv(file)

#     #close the browser
#     browser.close()

#     #return the dataframe so it can be used for visualization and data analysis
#     return df

In [5]:
# url = "https://www.bovada.lv/sports/baseball/futures-odd"
# file = "mlb_new_021224.csv"
# #return the response to a dataframe so that it can be used
# df_mlb = bovada_scrape(url, file)
# df_mlb.head(5)

ValueError: All arrays must be of the same length

In [11]:
def bovada_scrape_2(url, file):
        # Initiate selenium and open the URL
    browser = webdriver.Chrome(service=service, options=options)
    browser.get(url)
    browser.implicitly_wait(10)

    # Find all the market content sections
    market_contents = browser.find_elements(By.CLASS_NAME, 'score-container')

    # Initiating lists for the data we want
    market_names = []
    outcomes = []
    bet_prices = []

    for section in market_contents:
        # Extract the market name
        try:
            market_name_element = section.find_element(By.CLASS_NAME, 'market-name')
            market_name = market_name_element.text
        except NoSuchElementException:
            market_name = "No Market Name"  # Or handle this case as you see fit

        # Find all bet options (outcomes and prices) within this section
        bet_options = section.find_elements(By.CLASS_NAME, 'other')
        for option in bet_options:
            # Extract the outcome and bet price
            try:
                outcome_element = option.find_element(By.CLASS_NAME, 'outcomes')
                bet_price_element = option.find_element(By.CLASS_NAME, 'bet-price')

                outcome_text = outcome_element.text
                bet_price_text = bet_price_element.text

                market_names.append(market_name)
                outcomes.append(outcome_text)
                bet_prices.append(bet_price_text)
            except NoSuchElementException:
                # Skip if outcome or bet price is missing
                continue

    # Create a dataframe from the lists
    df = pd.DataFrame({'Market Name': market_names, 'Outcome': outcomes, 'Bet Price': bet_prices})

    # Handling of the DataFrame (e.g., saving, returning) goes here
    #save off the file, overwriting the existing
    df.to_csv(file, index = None)
    df = pd.read_csv(file)

    #close the browser
    browser.close()

    #return the dataframe so it can be used for visualization and data analysis
    return df

In [12]:
url = "https://www.bovada.lv/sports/baseball/futures-odd"
file = "mlb_new_021324.csv"
#return the response to a dataframe so that it can be used
df_mlb = bovada_scrape_2(url, file)
df_mlb.head(5)

,Market Name,Outcome,Bet Price
0,Winning League,National League Team,-125
1,Winning League,American League Team,-105
2,"MLB 2024 Regular Season ""Any"" Specials",40-40 Club - Any 2+ Players To Record 40+ Home...,110
3,"MLB 2024 Regular Season ""Any"" Specials",40-40 Club - Any 3+ Players To Record 40+ Home...,500
4,"MLB 2024 Regular Season ""Any"" Specials",Any Pitcher To Win The Regular Season AL Or NL...,1800


In [8]:
# browser = webdriver.Chrome(service=service, options=options)
# browser.get(url)
# browser.implicitly_wait(10)

# # Find all the market content sections
# market_contents = browser.find_elements(By.CLASS_NAME, 'score-container')
# for section in market_contents:
#     market_name_element = section.find_element(By.CLASS_NAME, 'market-name')
#     market_name = market_name_element.text
#     print(market_name)